In [3]:
import sqlite3

https://docs.python.org/3/library/sqlite3.html

In [4]:
con = sqlite3.connect("my_database.db")
cur = con.cursor()

### Создание таблицы

In [5]:
cur.execute(
    """CREATE TABLE table1
                (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                col1 VARCHAR(255),
                col2 date,
                col3 INTEGER
                )
    """
)

OperationalError: table table1 already exists

Выполнение запросов

In [6]:
cur.execute("INSERT INTO table1 (col1, col2, col3) VALUES ('one','2022-01-01',11)")
cur.execute("INSERT INTO table1 (col1, col2, col3) VALUES ('two','2022-01-02',22)")
cur.execute("INSERT INTO table1 (col1, col2, col3) VALUES ('three','2022-01-03',33)")
con.commit()

In [7]:
cur.execute("commit;")

OperationalError: cannot commit - no transaction is active

In [8]:
cur.execute("SELECT * FROM table1")

In [9]:
for row in cur.execute("SELECT * FROM table1"):
    print(row)

(1, 'one', '2022-01-01', 11)
(2, 'two', '2022-01-02', 22)
(3, 'three', '2022-01-03', 33)
(4, 'one', '2022-01-01', 11)
(5, 'two', '2022-01-02', 22)
(6, 'three', '2022-01-03', 33)
(7, 'one', '2022-01-01', 11)
(8, 'two', '2022-01-02', 22)
(9, 'three', '2022-01-03', 33)


In [10]:
cur.execute("SELECT * FROM table1")

In [11]:
cur.fetchone()

(1, 'one', '2022-01-01', 11)

In [12]:
cur.fetchone()

(2, 'two', '2022-01-02', 22)

In [13]:
cur.fetchone()

(3, 'three', '2022-01-03', 33)

In [14]:
cur.fetchone()

(4, 'one', '2022-01-01', 11)

In [15]:
cur.execute("SELECT * FROM table1")
cur.fetchall()

[(1, 'one', '2022-01-01', 11),
 (2, 'two', '2022-01-02', 22),
 (3, 'three', '2022-01-03', 33),
 (4, 'one', '2022-01-01', 11),
 (5, 'two', '2022-01-02', 22),
 (6, 'three', '2022-01-03', 33),
 (7, 'one', '2022-01-01', 11),
 (8, 'two', '2022-01-02', 22),
 (9, 'three', '2022-01-03', 33)]

Закрытие подключения

In [16]:
con.close()

In [17]:
cur.execute("SELECT * FROM table1")
cur.fetchall()

ProgrammingError: Cannot operate on a closed database.

In [18]:
con = sqlite3.connect("my_database.db")
cur = con.cursor()

In [19]:
cur.execute("SELECT * FROM table1")
cur.fetchall()

[(1, 'one', '2022-01-01', 11),
 (2, 'two', '2022-01-02', 22),
 (3, 'three', '2022-01-03', 33),
 (4, 'one', '2022-01-01', 11),
 (5, 'two', '2022-01-02', 22),
 (6, 'three', '2022-01-03', 33),
 (7, 'one', '2022-01-01', 11),
 (8, 'two', '2022-01-02', 22),
 (9, 'three', '2022-01-03', 33)]

In [20]:
con.close()

Работа в оперативной памяти

In [21]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

In [22]:
cur.execute(
    """CREATE TABLE table1
                (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                col1 VARCHAR(255),
                col2 date,
                col3 INTEGER
                )
              """
)

In [23]:
values = [
    ("one", "2022-01-01", 11),
    ("two", "2022-01-02", 22),
    ('three', '2022-01-03', 33),
]
cur.executemany("insert into table1 (col1, col2, col3) VALUES  (?, ?, ?)", values)
con.commit()

In [24]:
cur.execute("SELECT * FROM table1")
cur.fetchall()

[(1, 'one', '2022-01-01', 11),
 (2, 'two', '2022-01-02', 22),
 (3, 'three', '2022-01-03', 33)]

Подключение к бд с помощью Pandas

In [26]:
import pandas as pd

https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html

In [27]:
pd.read_sql("SELECT * FROM table1", con)

,id,col1,col2,col3
0,1,one,2022-01-01,11
1,2,two,2022-01-02,22
2,3,three,2022-01-03,33


In [28]:
data = pd.read_sql("SELECT * FROM table1", con)

In [29]:
data.head()

,id,col1,col2,col3
0,1,one,2022-01-01,11
1,2,two,2022-01-02,22
2,3,three,2022-01-03,33


In [30]:
data.columns

Index(['id', 'col1', 'col2', 'col3'], dtype='object')

In [31]:
data.dtypes

id       int64
col1    object
col2    object
col3     int64
dtype: object

In [32]:

data = pd.read_sql("SELECT * FROM table1", con, parse_dates=["col2"])

In [33]:
data.head()

,id,col1,col2,col3
0,1,one,2022-01-01,11
1,2,two,2022-01-02,22
2,3,three,2022-01-03,33


In [34]:
data.dtypes

id               int64
col1            object
col2    datetime64[ns]
col3             int64
dtype: object

In [35]:
data = pd.read_sql(
    "SELECT * FROM table1", con, parse_dates={"col2": {"format": "%Y-%m-%d"}}
)

https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

In [36]:
data.head()

,id,col1,col2,col3
0,1,one,2022-01-01,11
1,2,two,2022-01-02,22
2,3,three,2022-01-03,33


In [37]:
data.dtypes

id               int64
col1            object
col2    datetime64[ns]
col3             int64
dtype: object

In [38]:
data["day"] = data["col2"].apply(lambda x: x.day)
data["month"] = data["col2"].apply(lambda x: x.month)
data["year"] = data["col2"].apply(lambda x: x.year)

In [39]:
data.head()

,id,col1,col2,col3,day,month,year
0,1,one,2022-01-01,11,1,1,2022
1,2,two,2022-01-02,22,2,1,2022
2,3,three,2022-01-03,33,3,1,2022


Загрузка данных в БД

In [40]:
table2 = pd.DataFrame(
    {
        "column1": [11, 22, 33],
        "column2": ["1", "2", "3"],
        "column3": ["2022-01-01", "2022-01-02", "2022-01-03"],
    }
)

In [41]:
table2

,column1,column2,column3
0,11,1,2022-01-01
1,22,2,2022-01-02
2,33,3,2022-01-03


In [42]:
table2.index

RangeIndex(start=0, stop=3, step=1)

In [43]:
table2.to_sql("table2", con)

3

In [44]:
data2 = pd.read_sql("SELECT * FROM table2", con)

In [45]:

data2.head()

,index,column1,column2,column3
0,0,11,1,2022-01-01
1,1,22,2,2022-01-02
2,2,33,3,2022-01-03


In [46]:
table2.to_sql("table2", con, index=False, if_exists="replace")

3

In [47]:
data2 = pd.read_sql("SELECT * FROM table2", con)

In [48]:
data2.head()

,column1,column2,column3
0,11,1,2022-01-01
1,22,2,2022-01-02
2,33,3,2022-01-03


In [49]:
data2.dtypes

column1     int64
column2    object
column3    object
dtype: object

In [50]:

data2 = pd.read_sql("SELECT * FROM table2", con, parse_dates="column3")

In [51]:
data2.head()

,column1,column2,column3
0,11,1,2022-01-01
1,22,2,2022-01-02
2,33,3,2022-01-03


In [52]:

data2.dtypes

column1             int64
column2            object
column3    datetime64[ns]
dtype: object